# **INF8460 A20 Project: Open-domain questions answering**

<br>

Equipe 8:


*   Cedric Sadeu (Glove, ranking with classification)
*   Mamoudou Sacko (pretraitement + TF-IDF, cosine ranking)
*   Oumayma Messoussi (PCP Bert, ML/DL for ranking)

<br>

---

<br>

In [2]:
!pip install sent2vec transformers pytorch-pretrained-bert


     |████████████████████████████████| 1.4MB 13.1MB/s 
     |████████████████████████████████| 133kB 50.5MB/s 
     |████████████████████████████████| 2.9MB 50.4MB/s 
     |████████████████████████████████| 890kB 56.5MB/s 
     |████████████████████████████████| 133kB 54.9MB/s 
     |████████████████████████████████| 6.9MB 54.2MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=dbb21b69620dfa325376f728f7a8ae23d11f468ed02eac4ea76874c920f07a9b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
ERROR: botocore 1.19.26 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


---

<br>

## **1. Map Google**


In [3]:

from google.colab import drive
drive.mount('/content/drive')
!ls '/content/drive/My Drive/Colab Notebooks/INF8460/Project/'

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/INF8460/Project/')


Mounted at /content/drive
bert.py		      data		inf8460_projet_A20_equipe8_clean.ipynb
correction_script.py  data_handling.py	output
create_embeddings.py  hub.py		__pycache__


---

<br>

## **2. Import**

In [4]:
import torch
import hub

import importlib
importlib.reload(hub)
from hub import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


---

<br>

## **3. Load Data**


In [5]:
corpus_data = load_data("corpus", 0)
vals_data = load_data("test", force_refresh = 0)

corpus_ids = corpus_data[0]
vals_ids = corpus_data[0]

corpus_paragraphs = corpus_data[1]
vals_questions = vals_data[1]

---

<br>

## **4. Prepoc Data**

In [6]:
corpus_text = prepoc_data(corpus_paragraphs)
vals_text = prepoc_data(vals_questions)

---

<br>

## **5. Representation**


In [7]:
representation_name = "tfidf"
corpus_emmbeding , vals_emmbeding = get_emmbending(corpus_text, vals_text, representation_name)


---

<br>

## **6. Ordonnancement**

In [ ]:
ranking_list = get_ranking_list(corpus_ids, corpus_emmbeding, vals_ids, vals_emmbeding, metrique=cosine, top=25)

[10830, 52483, 47824, 10823, 10814, 10821, 67042, 10833, 10826, 63154, 10827, 10837, 10832, 10815, 10839, 10824, 10829, 12466, 10825, 10828, 10820, 10838, 10835, 16528, 10819] [0.4610968808840715, 0.5321410004950893, 0.5508049004855129, 0.5511164041814345, 0.556268858903005, 0.5703590447010061, 0.5717612795051185, 0.5744885833432647, 0.5779364548336041, 0.578451534431152, 0.5836160451069012, 0.5881126567383779, 0.5937595250334385, 0.5954957241758947, 0.5964374107146693, 0.6156275467568522, 0.6184186464198156, 0.621891485541715, 0.623038796080583, 0.6356399101575632, 0.6393972034200908, 0.6411073463441053, 0.6447673732283893, 0.649748950765656, 0.6502274372891159]
[18259, 6915, 6890, 6908, 71261, 6920, 3604, 6916, 3580, 6921, 71936, 6905, 6910, 79723, 3652, 6895, 6909, 63435, 6914, 6897, 80542, 3581, 6919, 6907, 3610] [0.46477266438000653, 0.5011683964117247, 0.5252343278850327, 0.5405081506660926, 0.5441634396710132, 0.5567696591976505, 0.5618531236151192, 0.5639015196895713, 0.5801779

/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24] [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
[31609, 60511, 74693, 45381, 53622, 71910, 71437, 47099, 43741, 49277, 58895, 22481, 61763, 39978, 64572, 24833, 45617, 56512, 16835, 62876, 56855, 20680, 56257, 30884, 49739] [0.6210080377421388, 0.6979237172280491, 0.7028158288921775, 0.7060910751470109, 0.7146431933565061, 0.7213337075715013, 0.7350898963381305, 0.7384287683621538, 0.7504151015124102, 0.7562957206113492, 0.7575308937028643, 0.7584399180103144, 0.7612471178993739, 0.7637149975950288, 0.7650929872633573, 0.7674784308140021, 0.7707122572080587, 0.7733735416469153, 0.7808999852018101, 0.7813616911099086, 0.7846598983383964, 0.7887226364270593, 0.7887226364270593, 0.793012814662967, 0.793012814662967]
[18, 6, 53122, 1, 9, 46880, 22, 7, 21, 25689, 29399, 32, 0, 13665, 76897, 5689, 36, 29610, 53401, 74625, 537

In [ ]:
import io
import os
import pandas as pd


output_path = "/content/drive/My Drive/Colab Notebooks/INF8460/Project/output"
TOP_N = 20
s = ";"
id = []
top_n = []
top_10 = []

for id_question in ranking_list:
  id.append(id_question) 
  top_n.append( s.join(str(v) for v in ranking_list[id_question][:TOP_N]) )
  top_10.append( s.join(str(v) for v in ranking_list[id_question][:10]) )

passage_submission = {'id': id, 'top_n': top_n, "top_10" : top_10 }
df = pd.DataFrame(passage_submission, columns= list(passage_submission.keys()))
display(df)
df.to_csv (os.path.join(output_path, "passage_submission.csv"), index = False, header=True)



---

<br>

## **7. Answers**

In [ ]:


from transformers import pipeline
nlp = pipeline("question-answering")
answers = get_answers(nlp, corpus_ids, corpus_paragraphs, vals_ids, vals_questions, ranking_list)



Answer: 'Hussain Sagar lake', score: 0.8545, start: 49, end: 67
Answer: 'GHMC area', score: 0.1317, start: 140, end: 149
Answer: 'Raffles Millennium International', score: 0.2488, start: 735, end: 767
Answer: 'Pritzker Prize', score: 0.3054, start: 676, end: 690
Answer: 'Heritage buildings', score: 0.9107, start: 0, end: 18
Answer: 'Bond Hall', score: 0.2587, start: 159, end: 168
Answer: 'private dwellings', score: 0.1265, start: 758, end: 775
Answer: 'Charminar', score: 0.523, start: 67, end: 76
Answer: 'private dwellings , such as dorms and rooms', score: 0.2916, start: 855, end: 898
Answer: 'Hyderabad - Bangalore', score: 0.0283, start: 531, end: 552
Answer: 'Hyderabad - Bangalore', score: 0.0283, start: 531, end: 552
Answer: 'Lal Bahadur Shastri Stadium', score: 0.1508, start: 647, end: 674
Answer: 'Rashtrapati Nilayam', score: 0.4698, start: 1197, end: 1216
Answer: 'the ruins of Golconda fort', score: 0.3441, start: 196, end: 222
Answer: 'City Scape', score: 0.9633, start: 4, end:

---

<br>

## **8. Analyse**

In [ ]:
import pandas as pd
df = pd.DataFrame(answers, columns= list(answers.keys()))
display(df)

,What is the oldest piece of Qutb Shahi architecture in Hyderabad?,What was incorporated to the united states in the first half of the 18th century?,What is a popular grain source for brewing beer in Mexico?,On what digital platform were contestants performances released to after airing on American Idol?,Which Democratic candidate was the first to win 40% of the vote in Alaska since 1946?,Who were the two most powerful nations in the 1860's?,Kanye has stated that members from which rap group influenced his early style?,What is attested as a self-identifier in Iranian languages?,What did this menace undermine?,What sub-group of literature emerged in the 21st century?,Who are among the most senior members of the Catholic Church?,What dynasty ruled Armenia in the 10th century?,How many Jews lived in the Bronx in 2011?,What can the Western Han dynasty also be called?,Where was the smallest camp established?,During the sparging process what do startches turn into?,"The first president, George Washington, took office in what year?","In Whitehead's cosmology, what are the only things that fundamentally exist?",What did several British politicians suggest doing in the 19th century?,What is one design element of Gothic construction that is always terminated by small pinnacles?,What does a behaviorial component focus on when defining sexual orientation?,What's the estimated population of Southampton?,What are castros?,What is the style of the saucer domes in the Belgian Suite?,Where did the tradition of seeing Virgil as a great magician last the longest?
0,"{'score': 0.0006140831392258406, 'start': 59, ...","{'score': 1.6595748775216634e-06, 'start': 133...","{'score': 0.030791986733675003, 'start': 403, ...","{'score': 3.2708889193600044e-05, 'start': 128...","{'score': 0.00048178693396039307, 'start': 547...","{'score': 0.014903667382895947, 'start': 316, ...","{'score': 0.000123982536024414, 'start': 1220,...","{'score': 0.04186447709798813, 'start': 263, '...","{'score': 0.05502988025546074, 'start': 199, '...","{'score': 0.0776226818561554, 'start': 399, 'e...","{'score': 0.030848762020468712, 'start': 202, ...","{'score': 0.052462365478277206, 'start': 309, ...","{'score': 2.2423061807330669e-07, 'start': 111...","{'score': 1.9231515580031555e-06, 'start': 968...","{'score': 0.012541801668703556, 'start': 547, ...","{'score': 0.017634278163313866, 'start': 50, '...","{'score': 0.04148123785853386, 'start': 174, '...","{'score': 0.001852959394454956, 'start': 1126,...","{'score': 0.0199835654348135, 'start': 51, 'en...","{'score': 0.016130199655890465, 'start': 372, ...","{'score': 0.006269362289458513, 'start': 235, ...","{'score': 0.0036219083704054356, 'start': 186,...","{'score': 0.007150379009544849, 'start': 311, ...","{'score': 3.0496019007841824e-06, 'start': 604...","{'score': 2.2568690837942995e-05, 'start': 159..."
1,"{'score': 0.028345992788672447, 'start': 531, ...","{'score': 0.039663370698690414, 'start': 153, ...","{'score': 0.060409124940633774, 'start': 19, '...","{'score': 7.41023977752775e-05, 'start': 675, ...","{'score': 0.002412612084299326, 'start': 8, 'e...","{'score': 0.027850467711687088, 'start': 229, ...","{'score': 0.05535002052783966, 'start': 64, 'e...","{'score': 0.04300808161497116, 'start': 418, '...","{'score': 0.05813860520720482, 'start': 258, '...","{'score': 0.09109410643577576, 'start': 358, '...","{'score': 0.04792541638016701, 'start': 586, '...","{'score': 0.058251723647117615, 'start': 308, ...","{'score': 2.14823849091772e-05, 'start': 200, ...","{'score': 1.0861492228286806e-05, 'start': 793...","{'score': 0.012541801668703556, 'start': 547, ...","{'score': 0.01920369267463684, 'start': 1208, ...","{'score': 0.04223277419805527, 'start': 271, '...","{'score': 0.018793705850839615, 'start': 536, ...","{'score': 0.021485723555088043, 'start': 563, ...","{'score': 0.04106787592172623, 'start': 482, '...","{'score': 0.017729394137859344, 'start': 290, ...","{'score': 0.004505982156842947, 